# Coursework for MRI reconstruction (Autumn 2019)

In this tutorial, we provide the data loader to read and process the MRI data in order to ease the difficulty of training your network. By providing this, we hope you focus more on methodology development. Please feel free to change it to suit what you need.

In [1]:
import h5py, os
from functions import transforms as T
from functions.subsample import MaskFunc
from scipy.io import loadmat
from torch.utils.data import DataLoader
import numpy as np
import torch
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
def show_slices(data, slice_nums, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)
        plt.axis('off')

In [3]:
class MRIDataset(DataLoader):
    def __init__(self, data_list, acceleration, center_fraction, use_seed):
        self.data_list = data_list
        self.acceleration = acceleration
        self.center_fraction = center_fraction
        self.use_seed = use_seed

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        subject_id = self.data_list[idx]
        return get_epoch_batch(subject_id, self.acceleration, self.center_fraction, self.use_seed)

In [4]:
def get_epoch_batch(subject_id, acc, center_fract, use_seed=True):
    ''' random select a few slices (batch_size) from each volume'''

    fname, rawdata_name, slice = subject_id  
    
    with h5py.File(rawdata_name, 'r') as data:
        rawdata = data['kspace'][slice]
                      
    slice_kspace = T.to_tensor(rawdata).unsqueeze(0)
    S, Ny, Nx, ps = slice_kspace.shape

    # apply random mask
    shape = np.array(slice_kspace.shape)
    mask_func = MaskFunc(center_fractions=[center_fract], accelerations=[acc])
    seed = None if not use_seed else tuple(map(ord, fname))
    mask = mask_func(shape, seed)
      
    # undersample
    masked_kspace = torch.where(mask == 0, torch.Tensor([0]), slice_kspace)
    masks = mask.repeat(S, Ny, 1, ps)

    img_gt, img_und = T.ifft2(slice_kspace), T.ifft2(masked_kspace)
    
    
    # perform data normalization which is important for network to learn useful features
    # during inference there is no ground truth image so use the zero-filled recon to normalize
    norm = T.complex_abs(img_und).max()
    if norm < 1e-6: norm = 1e-6
    
    # normalized data
    img_gt, img_und, rawdata_und = img_gt/norm, img_und/norm, masked_kspace/norm
    
#    img_gt = T.center_crop(T.complex_abs(img_gt), [320, 320]).unsqueeze(1)
#    img_und = T.center_crop(T.complex_abs(img_und), [320, 320]).unsqueeze(1)
#     rawdata_und = T.center_crop(T.complex_abs(rawdata_und), [320, 320]).unsqueeze(1)
#     norm = T.center_crop(T.complex_abs(norm), [320, 320]).unsqueeze(1)
#     masks.T.center_crop(T.complex_abs(masks), [320, 320]).unsqueeze(1)    

    img_gt = T.center_crop(T.complex_abs(img_gt), [320, 320])
    img_und = T.center_crop(T.complex_abs(img_und), [320, 320])
        
    return img_gt.squeeze(0), img_und.squeeze(0)


In [5]:
def load_data_path(train_data_path, val_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_and_val = ['train', 'val']
    data_path = [train_data_path, val_data_path]
      
    for i in range(len(data_path)):

        data_list[train_and_val[i]] = []
        
        which_data_path = data_path[i]
    
        for fname in sorted(os.listdir(which_data_path)):
            
            subject_data_path = os.path.join(which_data_path, fname)
                     
            if not os.path.isfile(subject_data_path): continue 
            
            with h5py.File(subject_data_path, 'r') as data:
                num_slice = data['kspace'].shape[0]
                
            # the first 5 slices are mostly noise so it is better to exlude them
            data_list[train_and_val[i]] += [(fname, subject_data_path, slice) for slice in range(5, num_slice)]
    
    return data_list    

In [6]:
class AlexNet(nn.Module):

    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1), #320/320
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), #320/320
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # 320/320
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), #320/320
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # 320/320
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, kernel_size=3, padding=1),  # 320/320
            
        )

    def forward(self, x):
        x = self.features(x)
        #x = nn.functional.sigmoid(x)
        #x = x * 255
        #x = x.type(torch.cuda.int32)
        return x

In [7]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [8]:
from skimage.measure import compare_ssim 
def ssim(gt, pred):
    """ Compute Structural Similarity Index Metric (SSIM). """
    return compare_ssim(
        gt.transpose(1, 2, 0), pred.transpose(1, 2, 0), multichannel=True, data_range=gt.max()
    )

In [9]:


if __name__ == '__main__':
    
    data_path_train = '/tmp/NC2019MRI/train'
    data_path_val = '/tmp/NC2019MRI/train'
    data_list = load_data_path(data_path_train, data_path_val) # first load all file names, paths and slices.
    
    acc = 8
    cen_fract = 0.04
    seed = False # random masks for each slice 
    num_workers = 12 # data loading is faster using a bigger number for num_workers. 0 means using one cpu to load data
    
    lr = 6e-3
    
    network = UNet(1,1)
    network.to('cuda:0') #move the model on the GPU
    mse_loss = nn.MSELoss().to('cuda:0')
    
    optimizer = optim.Adagrad(network.parameters(), lr=lr)
    
    #create data loader for training set. It applies same to validation set as well
    train_dataset = MRIDataset(data_list['train'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=5, num_workers=num_workers) 
    

    for epoch in range(2):
        for iteration, sample in enumerate(train_loader):
        
            img_gt, img_und = sample
        
            img_gt = img_gt.unsqueeze(1).to('cuda:0')
            img_und = img_und.unsqueeze(1).to('cuda:0')

            output = network(img_und)       #feedforward
            #output = output.squeeze(1).cpu().detach().numpy()

            loss = mse_loss(output, img_gt)
            #loss = torch.tensor(ssim(img_gt, output)).to('cuda:0')
            #print(loss.item())
            optimizer.zero_grad()       #set current gradients to 0
            loss.backward()      #backpropagate
            optimizer.step()     #update the weights

            print(loss.item(), "  ")
        

        
        # stack different slices into a volume for visualisation
#         A = masks[...,0].squeeze()
#         B = torch.log(T.complex_abs(rawdata_und) + 1e-9).squeeze()
#         C = T.complex_abs(img_und).squeeze()
#         D = T.complex_abs(img_gt).squeeze()
#         all_imgs = torch.stack([A,B,C,D], dim=0)

#         # from left to right: mask, masked kspace, undersampled image, ground truth
#         show_slices(all_imgs, [0, 1, 2, 3], cmap='gray')
#         plt.pause(1)

#         if iteration >= 0: break  # show 4 random slices
        

0.15101312100887299   
1.4425983428955078   
1.4402141571044922   
0.15985983610153198   
0.1011737659573555   
0.050378698855638504   
0.02328074537217617   
0.032161399722099304   
0.0170602947473526   
0.015053101815283298   
0.014114351943135262   
0.014127738773822784   
0.009700944647192955   
0.014771457761526108   
0.012811270542442799   
0.011615210212767124   
0.011751163750886917   
0.01390205230563879   
0.012319824658334255   
0.010012771002948284   
0.0144766541197896   
0.021094605326652527   
0.013093112036585808   
0.011681227013468742   
0.01188849750906229   
0.0196065753698349   
0.011501949280500412   
0.01778835989534855   
0.010597818531095982   
0.016620924696326256   
0.016758093610405922   
0.01882094517350197   
0.009998257271945477   
0.013137007132172585   
0.0184067040681839   
0.011674619279801846   
0.01126742921769619   
0.009039469063282013   
0.012987323105335236   
0.01338907890021801   
0.012886724434792995   
0.018462078645825386   
0.0090036662295

0.011412256397306919   
0.009716465137898922   
0.011013743467628956   
0.012938193045556545   
0.011845782399177551   
0.017424223944544792   
0.015848226845264435   
0.011568063870072365   
0.00927746668457985   
0.010722549632191658   
0.009423954412341118   
0.0110224150121212   
0.013058441691100597   
0.009189977310597897   
0.008934439159929752   
0.01279452908784151   
0.011164965108036995   
0.021292496472597122   
0.016416218131780624   
0.015629546716809273   
0.011821306310594082   
0.011355185881257057   
0.009776846505701542   
0.010446428321301937   
0.013928921893239021   
0.012903185561299324   
0.018853118643164635   
0.018846584483981133   
0.009565931744873524   
0.008369894698262215   
0.008496559225022793   
0.011136000044643879   
0.012922286055982113   
0.014183850958943367   
0.01850060373544693   
0.011567057110369205   
0.008677320554852486   
0.014141114428639412   
0.015523144043982029   
0.011527773924171925   
0.00902960542589426   
0.014668351039290428  

0.00686990050598979   
0.010232379660010338   
0.012048578821122646   
0.009433319792151451   
0.011265520006418228   
0.007917865179479122   
0.010329618118703365   
0.01343134418129921   
0.012784476391971111   
0.00983603298664093   
0.013675522990524769   
0.010712148621678352   
0.012072982266545296   
0.014194424264132977   
0.012227555736899376   
0.010113265365362167   
0.009963054209947586   
0.013742635026574135   
0.011592205613851547   
0.009819810278713703   
0.008122277446091175   
0.01259947195649147   
0.008532899431884289   
0.013681931421160698   
0.011103793978691101   
0.008061419241130352   
0.0089411661028862   
0.011536496691405773   
0.007883968763053417   
0.009655972942709923   
0.008786603808403015   
0.014227470383048058   
0.009009536355733871   
0.011202111840248108   
0.01058956515043974   
0.021737905219197273   
0.009955278597772121   
0.011109883897006512   
0.01463443972170353   
0.01678716577589512   
0.009893014095723629   
0.011548561975359917   
0

In [10]:
image, gt = train_dataset[6]
image = image.unsqueeze(0).to('cuda:0')
image = image.unsqueeze(0)
#gt = gt.unsqueeze(0).to('cuda:0')
gt = gt.unsqueeze(0).numpy()
output = network(image)
output = output.squeeze(1).cpu().detach().numpy()
loss = torch.tensor(ssim(gt, output))
loss2 = torch.tensor(ssim(gt, image.squeeze(1).cpu().numpy()))
print(loss.item())
print(loss2.item())
#loss2 = mse_loss(output, gt)
len(train_dataset)

0.4205639660358429
0.40286433696746826


/bham/modules/roots/neural-comp/2019-20/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """


2134

In [11]:
e = []
a=[]
b=[]
i = 0
for i in range(0,len(train_dataset)):
    image, gt = train_dataset[i]
    image = image.unsqueeze(0).to('cuda:0')
    image = image.unsqueeze(0)
    #gt = gt.unsqueeze(0).to('cuda:0')
    gt = gt.unsqueeze(0).numpy()
    output = network(image)
    output = output.squeeze(1).cpu().detach().numpy()
    loss = torch.tensor(ssim(gt, output))
    loss2 = torch.tensor(ssim(gt, image.squeeze(1).cpu().numpy()))
    e.append(loss.item()-loss2.item())
    a.append(loss.item())
    b.append(loss2.item())
    if loss.item()-loss2.item() < 0:
        i+=1
print(np.nanmean(e))
print(np.nanmean(a))

/bham/modules/roots/neural-comp/2019-20/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """


0.058093959928229244
0.4255652363152848


In [10]:
image, gt = train_dataset[3]
image = image.unsqueeze(0).to('cuda:0')
image = image.unsqueeze(0)
gt = gt.unsqueeze(0).to('cuda:0')
gt = gt.unsqueeze(0)
output = network(image)
loss2 = mse_loss(output, gt)
print(loss2.item())

0.007775604259222746


In [ ]:
acc = 8
cen_fract = 0.04
seed = False # random masks for each slice 
num_workers = 12 # data loading is faster using a bigger number for num_workers. 0 means using one cpu to load data
    

if __name__ == '__main__':
    
    data_path_train = '/tmp/NC2019MRI/train'
    data_path_val = '/tmp/NC2019MRI/train'
    data_list = load_data_path(data_path_train, data_path_val) # first load all file names, paths and slices.
    
    acc = 8
    cen_fract = 0.04
    seed = False # random masks for each slice 
    num_workers = 12 # data loading is faster using a bigger number for num_workers. 0 means using one cpu to load data
    # create data loader for training set. It applies same to validation set as well
    train_dataset = MRIDataset(data_list['train'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=1, num_workers=num_workers) 
    

    a = [[],[]]
    for iteration, sample in enumerate(train_loader):
        img_gt, img_und, rawdata_und, masks, norm = sample
        img_gt = T.center_crop(T.complex_abs(img_gt), [320, 320]).unsqueeze(1)
        img_und = T.center_crop(T.complex_abs(img_und), [320, 320]).unsqueeze(1)
        a[0].append(img_und)
        a[1].append(img_gt)
    b = torch.cat(a[0][:])
    c = torch.cat(a[1][:])
train = torch.stack((b,c),dim=0)
del a
del b
del c
del train_loader
del train_dataset
train.shape

In [11]:
lr = 1e-3
    
network = AlexNet()
network.to('cuda:0') #move the model on the GPU
mse_loss = nn.MSELoss().to('cuda:0')
    
optimizer = optim.Adam(network.parameters(), lr=lr)
train_loader = DataLoader(train, shuffle=True, batch_size=1, num_workers=num_workers) 
for iteration, sample in enumerate(train_loader):
    #img_gt, img_und, rawdata_und, masks, norm = sample        
    
    output = network(img_und)       #feedforward
    print(output.shape)

    loss = mse_loss(output, img_gt)
    optimizer.zero_grad()       #set current gradients to 0
    loss.backward()      #backpropagate
    optimizer.step()     #update the weights
    print(loss.item(), "  ")
        
    i = 0
    j +=1
        
    if j%100 == 0:
        for row in range(0,320):
            for col in range(0,320):
                if output[0,0,row,col].item() == img_gt[0,0,row,col].item():

                        i +=1
        print(i, "\n \n")

ValueError: not enough values to unpack (expected 5, got 1)

In [12]:
a = torch.tensor([2])
a.astype(np.float64)

AttributeError: 'Tensor' object has no attribute 'astype'